In [2]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [4]:
model = ChatGoogleGenerativeAI(
    model = os.getenv("GEMINI_MODEL"),
    api_key = os.getenv("Google_API_KEY")
)

In [5]:
# create a state

class LLMState(TypedDict):
    question: str
    answer: str
    

In [6]:
def llm_qa(state: LLMState) -> LLMState:
    
    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f"Answer the following question{question}"

    # ask that question to the llm
    answer = model.invoke(prompt).content


    # update the answer in the state
    state['answer'] = answer

    return state

In [7]:
# create graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa',llm_qa)

# add edges
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

# compile
workflow = graph.compile()


# execute
intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

print(final_state)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details..


{'question': 'How far is moon from the earth?', 'answer': "The distance between the Earth and the Moon varies because the Moon's orbit is an ellipse, not a perfect circle.\n\n*   **Average Distance:** The average distance is about **384,400 kilometers (238,900 miles)**.\n*   **Closest Point (Perigee):** Around 363,104 kilometers (225,623 miles).\n*   **Farthest Point (Apogee):** Around 405,696 kilometers (252,088 miles)."}


In [8]:
# other way 

model.invoke("How far is moon from the earth?").content

"The distance between the Earth and the Moon isn't constant, as the Moon's orbit is elliptical.  Here's a breakdown:\n\n*   **Average Distance:** 384,400 kilometers (238,900 miles)\n\n*   **Closest (Perigee):** Around 363,104 kilometers (225,623 miles)\n\n*   **Farthest (Apogee):** Around 405,696 kilometers (252,088 miles)"